# Raster Data Analysis in Python

*2 hours*

---

## Introduction to `rasterio`

We're finally ready to start working with spatial file formats in Python! We'll introduce the library `rasterio`, which is capable of reading raster data files like GeoTIFF files.

In [ ]:
import rasterio as rio

neon = rio.open('http://files.ntsg.umt.edu/data/GIS_Programming/data/NEON_ortho.tif')
neon

A `rasterio` dataset has some helpful attributes, like most Python objects.

In [ ]:
neon.mode

In [ ]:
neon.count

This is a multi-band GeoTIFF, so it has multiple **indexes.**

In [ ]:
neon.indexes

It's possible for some datasets with multiple bands to have different datatypes for each band.

In [ ]:
neon.dtypes

**As we've seen, a raster can be represented as a multi-dimensional array...**

In [ ]:
(neon.height, neon.width)

In fact, `rasterio` builds on top of NumPy to represent the underlying raster data.

In [ ]:
neon.shape

**But there are two things that distinguish a raster dataset from a multi-dimensional array.**

One of those things is the **spatial projection,** also known as Spatial Reference System (SRS) or Coordinate Reference System (CRS).

In [ ]:
neon.crs

Where `32606` is the European Petroleum Survey Group (EPSG) code for UTM Zone 6. This numeric code might not mean much to us, but we can get more information about the CRS by looking at its **Well-Known Text (WKT)** representation:

In [ ]:
neon.crs.to_wkt()

We can look EPSG codes at the [website EPSG.io](https://epsg.io/32606); *or,* we could use [the `pyproj` library:](https://pyproj4.github.io/pyproj/stable/api/crs/crs.html)

In [ ]:
from pyproj import CRS

srs = CRS.from_user_input(32606)
srs.to_wkt()

Note that the above WKT string is different from the one given by `neon.crs.to_wkt()`; in general, we want to trust the CRS that's defined on the raster dataset we're working with.

---

## Defining the Spatial Reference System in Python

The projection tells us how a raster image should be displayed but it doesn't tell us where, on the surface of the Earth, it should be positioned, nor does it tell us to rotate or warp the image so that it looks correct.

**To completely describe a raster dataset in Python, we need two pieces of information:**

1. The coordinate reference system (CRS) or *projection* of the data; this describes how the flat image corresponds to the non-flat Earth.
2. The *affine transformation,* which describes how the raster's rows and columns line up with the geospatial coordinates, e.g., latitude and longitude.

**The second piece of information, the affine transformation, is obtained:**

In [ ]:
neon.transform

**What does this mean?** The affine transform always consists of 6 numbers:

```py
(pixel_width, row_rotation, x_min, col_rotation, -pixel_height, y_max)
```

1. `pixel_width` is the width of a raster pixel in the units used by the SRS, usually meters or degrees.
2. `row_rotation` describes how rows are oriented on the map; for North-up maps this is always zero.
3. `x_min` is the minimum X coordinate, for example, the minimum or west-most Longitude.
4. `col_rotation` is similar to `row_rotation` and is always zero for North-up maps.
5. `-pixel_height` is the *negative* height of a raster pixel, in the units used by the SRS.
6. `y_max` is the maximum Y coordinate, for example, the maximum or north-most Latitude.

[You can read more about the affine transform and its implementation in `rasterio` here.](https://github.com/rasterio/affine)

**So, in this example:**

In [ ]:
neon.transform

- `470000.0` is the minimum X coordinate (minimum Easting, given this is a UTM projection)
- `7228000.0` is the maximum Y coordinate (maximum Northing, given this is a UTM projection)
- `1.0` is the pixel width, i.e., 1 meter
- `-1.0` is the pixel height, also 1 meter. We'll discuss why it is a negative number.

An **affine transformation** is a kind of transformation that preserves lines and paralellism; in GIS, it's a way of mapping one 2D grid onto a different gridded coordinate system.

![](./assets/affine-transformation.jpg)

*Image from [GeeksForGeeks.org](https://www.geeksforgeeks.org/python-opencv-affine-traansformation/)*

For example, an affine transformation describes the change in perspective between a billboard seen at an angle and the original image.

![](./assets/affine-transformation2.png)

*Image from [Felipe Meganha](https://felipemeganha.medium.com/perspective-transformation-with-kornia-8bf86718adfd)*

Let's build some intuition about raster data arrays in Python. When we work with raster data in Python, we have to keep in mind that there are two coordinate systems: 

1. The spatial coordinate system (SRS), which describes where something is on our representation of the Earth;
2. The image coordinate system, which describes the location of a value within the array's data structure.

The image coordinate system, for a single-band raster image, consists of rows and columns. **One important thing to remember is that row-column values *increase* from top to bottom in the image coordinate system, whereas spatial coordinate (e.g., latitude) *decrease* in that same direction.**

![](./assets/coordinate-system-diagram.png)

So, with `neon.crs` and `neon.transform`, we have all the information needed to display the raster's rows and columns correctly on a map. But what about the raster data itself? How can we actually start working with the raster data values?

---

## Working with Raster Data in Python

As we've discussed, raster data in Python can be represented by NumPy arrays. The first step to working with or visualizing the data is to get the raster data into a NumPy array. This is done with the `read()` method of a `rasterio` dataset...

In [ ]:
# Read the first band of the raster dataset
arr = neon.read(1)
arr

In [ ]:
arr.shape

As expected, the raster has 1000 rows and 1000 columns, just as the `gdal.Dataset` reported.

In [ ]:
from matplotlib import pyplot
pyplot.imshow(arr)

This image shows part of a fire scare in central Alaska. The data values correspond to the surface *albedo,* which describes the fraction of incoming sunlight that is reflected by the Earth's surface. You can see that the fire scar has a much lower albedo, and appears darker, than the unburned areas at the bottom-left and top-right of the image. Two roads cutting through the area also appear brighter.

**One important thing to note when we're working with raster data as `numpy` arrays.**

- The rows of an array increase from top to bottom. This is different from most spatial coordinates, like latitude.
- The columns of an array increase from left to right, which is similar to spatial coordinates like longitude.

Therefore, the top-left value of an array is the north-west corner of our image.

In [ ]:
arr[0,0]

Because these are albedo data, the values fall between zero and one. We can ask `numpy` for the percentiles of the data, e.g., the 0th percentile (minimum), 50th percentile (median), and 100th percentile (maximum).

In [ ]:
import numpy as np

np.percentile(arr, (0, 50, 100))

Because the raster data are a `numpy` array, we can operate on them as if they contain any other kind of data. In addition, we usually don't need to think at all about the spatial coordinate system when we're working with the data.

In [ ]:
# Applying a stretch to the data
lower, upper = np.percentile(arr, (2, 98))

stretch = arr.copy()
stretch[stretch < lower] = lower
stretch[stretch > upper] = upper
pyplot.imshow(stretch)

---

## Writing Raster Data to a File

There are several reasons why you might prefer to use `rasterio` to read and write raster data files. For one, writing a raster data file can be done in fewer steps with `rasterio` than with `gdal`.

It begins with opening a dataset for writing, specifying the spatial attributes of the raster we want to write.

In [ ]:
import numpy as np

dset = rio.open(
    'rio_output.tif', 'w', driver = 'GTiff', height = 1000, width = 1000, 
    count = 3, dtype = np.uint8, crs = neon.crs, transform = neon.transform)

When a file is opened for writing with `rasterio`, the following keyword arguments are required:

- `driver`, e.g., `"GTiff"` for GeoTIFF output
- `width` and `height`, in pixels
- `count`, which is the number of bands
- `dtype`, which is the data type and can be specified by a `numpy` datatype

If we don't provide a `crs` or `transform` argument, we will get a warning, but not a fatal error. Here, I've just used the `crs` and `transform` attributes of the existing dataset.

Now that the file is open for writing and is set up with a spatial reference system, we can write data into the file. The `write()` method works similar to the `read()` method, where we specify the band number.

For example, we can write our stretched image to this file:

In [ ]:
dset.write(stretch, 1)

Finally, when we're done writing data, we close the file.

In [ ]:
dset.close()

We only wrote one band to the file, but this should give you the basic idea of how writing data with `rasterio` works.

---

## Multispectral Raster Data and Spectral Indices

We can read multiple bands from a `rasterio` dataset in one call to `read()`:

In [ ]:
rgb = neon.read((1, 2, 3))
rgb.shape

Here, the first axis is the "band" axis; it represents the three bands in this image: Red, Green, and Blue.

**Can we plot this image?** Well, `pyplot.imshow()` is very particular about what kinds of arrays it can plot. In general:

- It can plot any 2D array
- It can plot a 3D array *only* if the last axis represents three (3) bands, which get mapped to Red, Green, and Blue

For example, `pyplot.imshow()` throws an error if we try to plot `rgb`, because its last axis has `1000` elements instead of `3`:

In [ ]:
pyplot.imshow(rgb)

You may be tempted to `reshape()` the `rgb` array to get the correct shape...

In [ ]:
pyplot.imshow(rgb.reshape((1000, 1000, 3)))

**But, whoa, this is clearly wrong.** We forgot that when we use `reshape()` the elements of our array get shuffled around. We can't just swap axis sizes this way.

What we want to do instead is to *roll* the first axis (3 elements) towards the end of the array (after the axis with 1000 elements).

In [ ]:
rgb2 = np.rollaxis(rgb, 0, 3)
rgb2.shape

Now we can get a nice RGB image.

In [ ]:
pyplot.imshow(rgb2)

Another way of showing multiple image bands at once, particularly when an RGB color combination isn't meaningful, is to just plot each band separately. Here's a `matplotlib` code block to do just that:

In [ ]:
fig = pyplot.figure(figsize = (10, 8))
fig.subplots_adjust(wspace = 0.2) # More horizontal space between plots
for band in range(0, 3):
    # add_subplot(nrows, ncols, index, ...); index must be non-zero so we add 1
    ax = fig.add_subplot(1, 3, 1 + band, title = f'Band {band+1}')
    ax.imshow(rgb[band])

---

### Challenge: Python Raster Calculator

The Normalized Difference Greenness Index (Escadafal & Huete 1991) or NDGI is a variation on the Normalized Difference Vegetation Index (NDVI). It can be calculated when only visible bands (e.g., red, green, blue) are available:

$$
\text{NDGI} = \frac{G - R}{G + R}
$$

Where $R$ is the Red band value and $G$ is the Green band value.

**Calculate the NDGI using this multi-band raster and then plot the resulting image.** The bands of this NEON dataset are, in order: Red, Green, Blue.

**NOTE:** Because the above calculation involves a fraction and will return floating-point data, we should first convert our array to a floating-point data type:

In [ ]:
rgb = rgb.astype(np.float32)

---

## Reprojection and Resampling

`rasterio` also handles reprojection and resampling. What's the difference?

- **Resampling** gridded data means to re-calculate its values on a new grid, where the new grid may be of a different resolution and/or offset from the original grid.
- **Projection** (or re-projection) refers to the process of figuring out how spatial coordinates in one SRS should be converted to another, and how the data associated with those coordinates will be resampled in order to match. **Hence, projection of gridded data always involves resampling but resampling does not imply projection.**

If we want to reproject one dataset to match the SRS of another, `rasterio` is much easier to use than `gdal`.

In [ ]:
ds_noaa = rio.open('http://files.ntsg.umt.edu/data/GIS_Programming/data/NOAA_NCEP_CPC_gridded_deg0p5_2020-2021_mean.tiff')
ds_soc = rio.open('http://files.ntsg.umt.edu/data/GIS_Programming/data/SPL4CMDL_Vv6040_20220901_SOC_9km.tiff')

**For practice, let's project the soil organic carbon (SOC) data onto the half-degree, equirectangular grid of the NOAA data.** This will allow us to explore the relationship between SOC storage and mean annual temperature.

First, we must create a new dataset that will receive the reprojected SOC data. The SRS of this output dataset, of course, is the same as the NOAA surface temperature data: a half-degree equirectangular coordinate system, which means there will be 360 rows of latitude and 720 columns of longitude.

**There are two important things to note about the dataset I'm opening:**

- As before, I'm going to specify the `'MEM'` driver so that the result is stored in memory, only, and not written to a file on disk. 
- I'm also going to set the `mode` as `'w+'` which means "write first, then read;" this is because I will want to read from the file after writing the reprojected data to it. There's a similar flag `'r+'` which means "read first, then write," but I can't use that flag because the dataset doesn't already exist.

In [ ]:
result = rio.open(
    'temp.file', 'w+', driver = 'MEM', width = 720, height = 360, count = 1,
    dtype = np.int16, crs = ds_noaa.crs, transform = ds_noaa.transform)

When using the `reproject()` method in `rasterio`, we need to distinguish between "source" and "destination." The "source" is the raster we want to reproject; the "destination" is the raster (and SRS) that has the projection we want. That's why we created `result`, above, as our "destination" raster, with an equirectangular coordinate system.

**The `source` and `destination` arguments of `reproject()` must be either:**

- A `numpy.ndarray`, or
- A `rasterio.Band`, which can be accessed using the `rasterio.band()` function

Basically, I want to read data from Band 1 of `ds_soc` and write the reprojected data into Band 1 of the `result` dataset.

In [ ]:
from rasterio.warp import reproject, Resampling

reproject(
    source = rio.band(ds_soc, 1), 
    destination = rio.band(result, 1), 
    resampling = Resampling.bilinear)

Did it work? Let's take a look.

In [ ]:
soc_ll = result.read(1)
soc_ll.shape

In [ ]:
pyplot.figure(figsize = (10, 6))
pyplot.imshow(soc_ll, vmin = 0, vmax = 4000)

In [ ]:
result.read(1).shape

[You can read more on the `rasterio.warp` module here.](https://rasterio.readthedocs.io/en/latest/api/rasterio.warp.html)

### Resampling without Projection

Sometimes we want to resample raster data without changing the raster's spatial reference system. 

For example, because our SOC data are on an equal-area grid, we can resample that data onto grids with cell sizes that are multiples of the original cell size. For 9-km data, for instance, we can resample it onto a 36-km grid where each 36-km pixel is made up of 16 (4x4) 9-km pixels. This is often called *downsampling.*

- **Downsampling:** When our resampling goes from finer grain to coarser grain; that is, the resulting pixel size is larger than the original.
- **Upsampling:** When our resampling goes from coarser grain to finer grain; that is, the resulting pixel size is smaller than the original.

To do this, we call the dataset's `read()` method and specify an `out_shape` or output shape. To resample from 9 km to 36 km, we want to reduce the width and height by a factor of 4.

$$
\frac{36}{9} = 4
$$

Note that we use integer division with `//` because the output shape must specify whole numbers of pixels.

In [ ]:
soc_36km = ds_soc.read(
    out_shape = (ds_soc.height // 4, ds_soc.width // 4), resampling = Resampling.average)

The output is an array, and it has a first axis that enumerates the bands; this is a single-band raster, so the first axis has just one element.

In [ ]:
soc_36km.shape

In [ ]:
pyplot.figure(figsize = (10, 6))
pyplot.imshow(soc_36km[0], vmin = 0, vmax = 4000)

If we want to write this resample data to a file, we'll need to calculate a new affine `transform`.

In [ ]:
new_transform = ds_soc.transform * ds_soc.transform.scale(4, 4)
new_transform

In [ ]:
soc_36km.shape

In [ ]:
output = rio.open(
    'resampled.tif', 'w', width = 964, height = 406,
    count = 1, dtype = np.int16, crs = ds_soc.crs, transform = new_transform)
output.write(soc_36km[0], 1)
output.close()

---

## Break

*10 minute break for learners.*

---

## Applying Functions to Axes

Let's switch to a different raster dataset, one that has multiple time periods, where each band in the image is a different time period.

We'll continue working with the NOAA NCEP data we saw earlier. In this version, I subset the data to part of North America and calculated an annual average.

In [ ]:
import requests
import numpy as np

content = requests.get('http://files.ntsg.umt.edu/data/GIS_Programming/data/NOAA_NCEP_CPC_gridded_deg0p5_1948-2022_Africa_74x149x143.float32')
data = np.frombuffer(content.content, dtype = np.float32)\
    .reshape((74, 149, 143))

As a reminder, we can plot the first year (1948) of data by typing:

In [ ]:
from matplotlib import pyplot

pyplot.imshow(data[0])
pyplot.colorbar()

In our lab this week, we calculated the maximum surface temperature at each pixel, over time, by typing:

In [ ]:
max_temp = data.max(axis = 0)
pyplot.imshow(max_temp)

Recall that `numpy` arrays have methods like `data.min()` and `data.mean()`, as well, which take axis arguments.

**But what if we wanted to calculate something other than the minimum, maximum, mean, or median in surface temperatures over this 74-year period?** There's no obvious way to do this with the tools we already. We need a way of applying a custom function to an array.

For situations like this, we can use `numpy.apply_along_axis()`.

In [ ]:
max_temp2 = np.apply_along_axis(max, 0, data)
pyplot.imshow(max_temp2)

The arguments to `numpy.apply_along_axis()` are, in order:

- The function you want to apply
- The axis you want to apply the function over
- The array

For the second argument, it's helpful to remember how the `axis` argument works, from this diagram:

![](./assets/numpy-axis.jpg)

**When we write `np.apply_along_axis(max, 0, data)`, we are saying we want the function `max()` to be applied to slices along the 0th axis.** 

i.e., every time `max()` is called it receives a slice of the 0th axis which, in this case, is the years axis. You can prove this to yourself by trying out:

In [ ]:
np.apply_along_axis(lambda x: x.size, 0, data)

We can see from this example that `max()` receives a 74-year time series when it is called. This allows us to summarize interannual data quickly and easily!

For instance, where do we see the biggest range in inter-annual temperatures?

In [ ]:
rng = np.apply_along_axis(lambda x: np.max(x) - np.min(x), 0, data)
pyplot.imshow(rng, vmax = 12)
pyplot.colorbar()
pyplot.show()

But clearly, the most interesting thing we could do is to calculate trends.

In [ ]:
from scipy import stats

def linear_trend(array):
    # linregress(x, y) takes two arguments: y is regressed on x
    result = stats.linregress(np.arange(0, 74), array)
    return result[0] # Just the slope

trends = np.apply_along_axis(linear_trend, 0, data)

In [ ]:
pyplot.imshow(trends)
pyplot.colorbar()
pyplot.show()

**Both the range and trends maps, above, look a little weird, probably because these gridded temperatures are interpolated from station data, which can be sparse.** If were really interested in extrapolating the range or trend in temperatures, we should probably use a remote-sensing based product, instead. But this works well for educational purposes.

---

## Array Masks

We've seen some examples of how raster data can be in Python handled as `numpy` arrays. Arrays come in multiple data types and we often need to handle or combine different types of numbers. Above, I provided you some code to read array data stored in a binary format; that's not typically how we'd store raster data!

**More often, you're going to want to read-in a a GeoTIFF.** Here's an example GeoTIFF representing soil organic carbon (SOC) data from [the NASA Soil Moisture Active Passive (SMAP) Level 4 Carbon (L4C) product.](https://nsidc.org/data/SPL4CMDL/)

In [ ]:
from osgeo import gdal

ds_soc = gdal.Open('http://files.ntsg.umt.edu/data/GIS_Programming/data/SPL4CMDL_Vv6040_20220901_SOC_9km.tiff')
soc = ds_soc.ReadAsArray()
soc.shape

In [ ]:
pyplot.imshow(soc)

**This doesn't look right. What did we forget?**

It's always helpful to look at the raw data.

In [ ]:
soc

Ah, so we have a bunch of NoData values in our array. Our plotting library doesn't know the difference between the number we use to represent NoData (-9999) and any other number value. Because -9999 is such an extreme value, our plot's colorbar is stretched too thin; we can't make out any variation in the actual data values.

### Handling NoData Values

Obviously, we need a way of telling our plotting library to ignore NoData values.

In [ ]:
soc[soc == -9999] = np.nan

We're on the right track, but now we have a problem with `numpy`, because we can't store `np.nan` in an array with an integer data type.

In [ ]:
soc = soc.astype(np.float32)
soc[soc == -9999] = np.nan

pyplot.figure(figsize = (12, 8))
pyplot.imshow(soc, vmin = 1000, vmax = 4000)

If the above image looks blurry or like it has a lot of holes in the data over land, note that you can tell `pyplot` to plot the data differently... Nearest-neighbor interpolation will pick the nearest raster pixel for each pixel on your screen.

In [ ]:
pyplot.figure(figsize = (12, 8))
pyplot.imshow(soc, vmin = 1000, vmax = 4000, interpolation = 'nearest')

### Array Masks

Before talk about zonal statistics, we should talk about array masks. 

As we saw when we converted our NoData values to `np.nan`, we can query `numpy` arrays using conditional expressions like this:

In [ ]:
soc[soc == -9999]

In [ ]:
soc[soc > 4000]

In these examples, if there is no assignment operator (`=`) on the right-hand side, the values in the array that match the conditional expression are pulled out and printed to the screen.

Note that the values are returned in a predictable order but they don't have a specific shape.

In [ ]:
soc[soc > 4000].shape

Here, there are over 65,000 SOC values greater than 4000 grams of carbon per meter squared... But we got all of the values as a 1D array.

Sometimes, the position of certain values within an array is important. In such cases, we actually want to take the conditional expression out of the slicing `[]` notation and use it to create a boolean array:

In [ ]:
high_soc = soc > 4000
high_soc

In [ ]:
high_soc.shape == soc.shape

**These kinds of arrays can be called array masks or masking arrays,** because they can be used to mask out or filter an array's contents. You'll use these kinds of conditional expressions frequently, often in combination with `np.where` or `np.argwhere`.

In [ ]:
pyplot.figure(figsize = (10, 8))
pyplot.imshow(np.where(soc > 2000, soc, np.nan))

[You should know that `numpy` has support for something called a *masked array.*](https://numpy.org/doc/stable/reference/maskedarray.html). However, NumPy masked arrays can be slow to work with, especially when the array size is large, so I would recommend you avoid them. Once you're comfortable with `numpy` functions like `np.where()` and with boolean arrays, you'll never need NumPy masked arrays.

---

## More Resources

- GIS&T Body of Knowledge: [Python for GIS](https://gistbok.ucgis.org/bok-topics/python-gis)
- GIS&T Body of Knowledge: [GDAL/OGR and Geospatial Data IO Libraries](https://gistbok.ucgis.org/bok-topics/gdalogr-and-geospatial-data-io-libraries)
- [GDAL Python API documentation](https://gdal.org/api/python/osgeo.gdal.html)
- [GDAL-OGR Cookbook](https://pcjericks.github.io/py-gdalogr-cookbook/index.html)
- [Reprojection - rasterio documentation](https://rasterio.readthedocs.io/en/latest/topics/reproject.html)